In [ ]:
# Collect constants

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

# Text-based Indexes that we are going to query (from Notebook 01 and 02)
index   = "projection-index2"

# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"]    = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"]     = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"]    = os.environ["OPENAI_API_TYPE"]
MODEL                            = os.environ["COMPLETION432_DEPLOYMENT"]
COMPLETION_TOKENS                = 1000

top_search_vector_k              = 5

# the question to ask - enable just one
# QUESTION = "Tell me briefly what are the advantages of defining compositionality using MDL."
# QUESTION = "What is a meaning function?"
QUESTION = "Why is von Neumann is considered as one of fathers of modern computers?"

In [ ]:
# Azure AI Search query

import requests, json

# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params  = {'api-version': os.environ['AZURE_SEARCH_API_VERSION_NEW']} # NEW VERSION!!!

# search query payload
search_payload = {
  "count": "true",
  "speller": "lexicon",
  "queryLanguage": "en-us",

  "top": top_search_vector_k,
  "select": "id, name, title, location, chunk",
  "vectorQueries": [
    {
      "kind": "text",
      "k": top_search_vector_k,
      "fields": "chunkVector",
      "text": QUESTION
    }
  ]
}

r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)

In [ ]:
r.json()

In [ ]:
# Format Azure AI Search results as an ordered dictionary

from collections import OrderedDict

ordered_results = OrderedDict()

for result in r.json()['value']:
    if result['@search.score'] > 0.25:# Show answers that are at least 25% of the max possible score=1
        ordered_results[result['id']]={
            "title": result['title'],
            "name": result['name'],
            "location": result['location'],
            "index": index,
            "chunk": result['chunk'],
            "score": result['@search.score']
        }

In [ ]:
# From the ordered dictionary, build the "Documents" array, needed by langchain "qa_with_sources" chain

from langchain.docstore.document import Document

top_docs = []
for key,value in ordered_results.items():
    location = value["location"] + os.environ['BLOB_SAS_TOKEN'] # to create "citations"
    top_docs.append(Document(page_content=value["chunk"], metadata={"source": location}))

In [ ]:
# Prepare the Open AI deployment

from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=COMPLETION_TOKENS)

In [ ]:
# Extract the specific answer from the Documents array, using Open AI through Langchain

from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# Choose either "stuff" or "map_reduce", depending on how long the text is. Leave just one uncommented:
chain_type = "stuff" # the LLM is able to manage the promtp + request + response in a single call
# chain_type = "map_reduce" # the LLM needs to split the promtp + request in multiple calls

if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, return_intermediate_steps=True)
    
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})

In [ ]:
# Print the final result, including the citation(s)

from IPython.display import display, HTML, Markdown

if chain_type == "map_reduce":
    for step in response['intermediate_steps']:
        display(HTML("<b>Chunk Summary:</b> " + step))

display(HTML(f"<br/><br/><b>FINAL ANSWER with --{chain_type}-- chain type:</b>"))
display(Markdown(response['output_text']))

In [ ]:
# Azure AI Search query

import requests, json

# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_OPENAI_API_KEY']}
params  = {'api-version': '2023-12-01-preview'}

# search query payload
search_payload = {
    "messages": [
        {
            "role": "user",
            "content": QUESTION
        }
    ],
    "temperature": 0,
    "top_p": 1.0,    
    "max_tokens": 1000,
    "dataSources": [
        {            
            "type": "AzureCognitiveSearch",
            "parameters": {                
                "endpoint": os.environ['AZURE_SEARCH_ENDPOINT'],
                "key": os.environ['AZURE_SEARCH_KEY'],
                "indexName": index
            }
        }
    ]
}

r = requests.post(
    f"{os.environ['AZURE_OPENAI_ENDPOINT']}openai/deployments/{MODEL}/extensions/chat/completions",
    data=json.dumps(search_payload), headers=headers, params=params
)

print(r.json()['choices'][0]['message']['content'])

In [ ]:
r.json()